In [17]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import xml.dom.minidom
import json
import csv
import datetime as dt


In [96]:
myToken = 'NLOD'
myUrl = 'https://api.entur.io/stop-places/v1/read/stop-places?count=50000&transportModes=BUS'
head1 = {'Authorization': 'token {}'.format(myToken)}
response = requests.get(myUrl, headers=head1)
# Print raw text content of the response to check the format, this is how I found out that it´s xml
#print(response.headers['Content-Type'])



In [97]:
#print(response.content)

In [98]:

#print(response.text) 

In [99]:
data = response.json() #converts JSON to Python dictionary
#print(data)

In [100]:
#print(data[0].keys())  #show the keys so that you can filter later

In [101]:
keys_to_keep = ['id', 'created', 'name', 'longitude', 'latitude']

processed_data = []
for entry in data:
    # Extract the 'value' from 'name'
    entry['name'] = entry.get('name', {}).get('value', None) 
    
    # Extract longitude and latitude from centroid['location']
    if 'centroid' in entry and 'location' in entry['centroid']:
        location = entry['centroid']['location']
        entry['longitude'] = location.get('longitude', None)
        entry['latitude'] = location.get('latitude', None)
        
        
        del entry['centroid']  # Remove the entire centroid field
    
    # Keep only the specified keys
    filtered_entry = {key: entry.get(key, None) for key in keys_to_keep}
    processed_data.append(filtered_entry)

# Convert the processed data into a DataFrame
df = pd.DataFrame(processed_data)



In [113]:
#df.head(12)

In [103]:
df.shape

(50000, 5)

In [112]:
# Define the bounding box for Oslo
oslo_min_lat = 59.85
oslo_max_lat = 59.97
oslo_min_lon = 10.64
oslo_max_lon = 10.89

# Filter the DataFrame
df_oslo = df[
    (df['latitude'] >= oslo_min_lat) & 
    (df['latitude'] <= oslo_max_lat) & 
    (df['longitude'] >= oslo_min_lon) & 
    (df['longitude'] <= oslo_max_lon)
]

# Display the filtered DataFrame
#print(df_oslo)


In [110]:

df_oslo.shape

(437, 5)

In [111]:
df_oslo.to_csv("bus_stop_oslo", index=False)